<a href="https://colab.research.google.com/github/ev-greshkina/DataAnalyst-educational-course/blob/main/%D0%94%D0%97_%D0%BA_%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D0%B8_20_v_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Выполнение ДЗ

In [57]:
# Подключение библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# Импортировать классификатор
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import csv
import nltk
from nltk.corpus import stopwords

In [58]:
# Получение иходных данных
# Файл для обработки должен быть в папке в блокноте, размер большой для копиролвания на github,
# настроить получение файла с google диска не удалось.
with open('lenta-ru-news.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    df = pd.DataFrame(reader)

In [59]:
# Уменьшить количесвто записей в датасете, для уменьшения времени вычисления
df = df.sample(100000).copy()

In [60]:
# Подготовить список стоп-слов
sw = stopwords.words('russian')

In [61]:
# Удалить все строки, в которых отсутствует значение в ячейке topic
df = df.loc[df['topic'] != '']

In [62]:
%%time
# Предобработка данных
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.replace(r'\W', ' ')

# Определить признаки целевой переменной
vectorizer = TfidfVectorizer(stop_words=sw, max_features=1000)
X = vectorizer.fit_transform(df['text'])
y = df['topic']

# Получение топ 4 тем
top_categories = y.value_counts().nlargest(4).index.tolist()
filtered_data = df[df['topic'].isin(top_categories)]
print(top_categories)

# Разделить данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    vectorizer.transform(filtered_data['text']),
    filtered_data['topic'],
    test_size=0.2,
    random_state=42
)

# Создание обучающей модели
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

# Оценка модели
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

['Россия', 'Мир', 'Экономика', 'Спорт']
              precision    recall  f1-score   support

         Мир       0.82      0.86      0.84      3382
      Россия       0.82      0.86      0.84      3966
       Спорт       0.98      0.89      0.93      1668
   Экономика       0.88      0.79      0.83      2012

    accuracy                           0.85     11028
   macro avg       0.87      0.85      0.86     11028
weighted avg       0.85      0.85      0.85     11028

CPU times: user 8min 48s, sys: 1.16 s, total: 8min 49s
Wall time: 8min 58s
